In [ ]:
import concurrent.futures
import json
import os
import requests
from bs4 import BeautifulSoup as bs
import logging
import time
if not os.path.exists('./data'):
    os.makedirs('./data')

In [ ]:
def getlink(page,linkset,):
    res=requests.get("http://www.ipeen.com.tw/search/taiwan/000/1-100-0-0/?baragain=1&p=1&p="+str(page))
    if res.status_code==200:
        soup=bs(res.text,'lxml')
        res.close()
        dienlist=soup.select("article.serMain > div.result > section > article.serItem > div > h3")#找尋店家資訊
        dienlist=[dien for dien in dienlist  if not dien.select_one("span.status")]#去除搬遷或關店狀態的店家
        dienlist=["http://www.ipeen.com.tw"+dien.select_one("a")["href"] for dien in dienlist 
                  if dien.select_one("a")["data-action"]!="ad_shop"]#去除廣告店家，留下網址
        linkset.update(dienlist)#將該頁的店list加入set中(順便去除重複)
#         if i%10==0:#每十頁存一次
#             print(len(linkset))
#             websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in linkset}
#             with open("./data/linkset.json","w") as f:
#                 json.dump(websitedict,f)
    else:#沒連線成功也存
        print("status_code="+str(res.status_code))
        res.close()
        print(len(linkset))
#         websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in linkset}
#         with open("./data/linkset.json","w") as f:
#             json.dump(websitedict,f)
#     websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in linkset}
    return dienlist

In [ ]:
res=requests.get("http://www.ipeen.com.tw/search/taiwan/000/1-100-0-0/?baragain=1&p=1&p=1")
soup=bs(res.text,"lxml")
res.close()
page=int(soup.select("#search > article > hgroup > h2 > b")[0].text)//15
print(str(page)+"pages")

WebsiteSet=set()
start = time.time()
i=0
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    jobInfo = [executor.submit(getlink, page, WebsiteSet,i) for page in range(1,page+1)]
    for future in concurrent.futures.as_completed(jobInfo):
        try:
            data = future.result()
            WebsiteSet.update(data)
        except Exception as e:
            print(e)
            print("zzz")
            pass
        
end = time.time()  
print("-----------------")
print(end-start)

websitedict={Website.split("-",1)[1]:Website.split("-",1)[0] for Website in WebsiteSet}
with open("./data/linkset.json","w") as f:
    json.dump(websitedict,f)

In [ ]:
with open("./data/linkSet.json") as f:
    x=json.load(f)

In [3]:
with open("./data/beginpage.txt","w") as f:
    f.write(str(30))


In [10]:
with open("./data/beginpage.txt") as f:
    page=int(f.read())

In [11]:
page

30